## Verilerin Lineer Regresyon İle Tahmin Edilmesi

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Veri setini eğitim ve test olarak ayırmak için
from sklearn.pipeline import Pipeline # İş akışlarını yönetmek için
from sklearn.preprocessing import StandardScaler, OneHotEncoder # Veriyi ölçeklendirmek ve kategorik verileri dönüştürmek için
from sklearn.compose import ColumnTransformer # Farklı sütunlara farklı ön işleme adımları uygulamak için
from sklearn.linear_model import LinearRegression # Lineer regresyon modelini kullanmak için
from sklearn.metrics import mean_squared_error, r2_score # Model performansını değerlendirmek için

In [20]:
df = pd.read_csv('hepsiemlak_cleaned_final.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7652 entries, 0 to 7651
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          7652 non-null   object
 1   district      7652 non-null   object
 2   neighborhood  7652 non-null   object
 3   room          7652 non-null   int64 
 4   livingroom    7652 non-null   int64 
 5   area          7652 non-null   int64 
 6   age           7652 non-null   int64 
 7   floor         7652 non-null   int64 
 8   price         7652 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 538.2+ KB
None


In [21]:
# Nümerik değişkenlerin minimum, maximum ve çeyreklik değerlerinin bulunması
df['city'] = df['city'].astype('category') # city sütununu kategorik veri tipine dönüştür
df['district'] = df['district'].astype('category') # district sütununu kategorik veri tipine dönüştür
df['neighborhood'] = df['neighborhood'].astype('category') # neighborhood sütununu kategorik veri tipine dönüştür
df['room'] = df['room'].astype('int') # room sütununu tamsayı veri tipine dönüştür
df['livingroom'] = df['livingroom'].astype('int') # livingroom sütununu tamsayı veri tipine dönüştür
df['area'] = df['area'].astype('int') # area sütununu tamsayı veri tipine dönüştür
df['age'] = df['age'].astype('int') # age sütununu tamsayı veri tipine dönüştür
df['floor'] = df['floor'].astype('int') # floor sütununu tamsayı veri tipine dönüştür
df['price'] = df['price'].astype('int') # price sütununu tamsayı veri tipine dönüştür

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7652 entries, 0 to 7651
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          7652 non-null   category
 1   district      7652 non-null   category
 2   neighborhood  7652 non-null   category
 3   room          7652 non-null   int32   
 4   livingroom    7652 non-null   int32   
 5   area          7652 non-null   int32   
 6   age           7652 non-null   int32   
 7   floor         7652 non-null   int32   
 8   price         7652 non-null   int32   
dtypes: category(3), int32(6)
memory usage: 307.5 KB
None


In [23]:
# Veri ön işleme adımlarını tanımlama
categorical_features = ['city', 'district', 'neighborhood'] # Kategorik değişkenler
numerical_features = ['room', 'livingroom', 'area', 'age', 'floor'] # Nümerik değişkenler

In [24]:
# Veri ön işleme adımlarını tanımlama
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features), # Nümerik değişkenler için ölçeklendirme
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # Kategorik değişkenler için one-hot encoding
])

In [25]:
X = df.drop('price', axis=1) # Özellikler (bağımsız değişkenler)
y = df['price'] # Hedef değişken (bağımlı değişken)

In [26]:
# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Veri setini eğitim ve test olarak ayırma # random_state=42 sabit bir bölme sağlar

In [27]:
# Model tanımlaması
model = Pipeline([
    ('preparation', full_pipeline), # Ön işleme adımları
    ('model', LinearRegression()) # Model tanımlaması
])

In [28]:
model.fit(X_train, y_train) # Modeli eğitim verisi ile eğitme

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['room', 'livingroom', 'area',
                                                   'age', 'floor']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'district',
                                                   'neighborhood'])])),
                ('model', LinearRegression())])

In [29]:
y_pred = model.predict(X_test) # Test verisi ile tahmin yapma
mse = mean_squared_error(y_test, y_pred) # Ortalama kare hata hesaplama
rmse = np.sqrt(mse) # Karekök ortalama kare hatayı hesaplama
r2 = r2_score(y_test, y_pred) # R^2 skorunu hesaplama

In [30]:
print(f"MSE: {mse}") # Ortalama kare hata
print(f"RMSE: {rmse}") # Karekök ortalama kare hata
print(f"R^2: {r2}") # R^2 skoru

MSE: 58503472.185072355
RMSE: 7648.756250860159
R^2: 0.5006983969561993


In [31]:
np.set_printoptions(threshold=np.inf)  # Tüm elemanları göster
features_importances = model.named_steps['model'].coef_ # Özelliklerin önem derecelerini alma
print(len(features_importances)) # Özelliklerin önem derecelerinin uzunluğunu yazdırma
print(features_importances) # Özelliklerin önem derecelerini yazdırma

1889
[ 1.43251542e+02  0.00000000e+00  4.86987899e+03 -3.57831405e+03
  8.72840208e+02 -2.62856988e+03  2.34994074e+03 -2.93838543e+03
 -8.64360199e+02  4.40012035e+03  3.47249188e+03  4.72221485e+03
  1.86266378e+03  3.17950125e+03  1.90531553e+03 -1.76124402e+04
 -2.31752323e+02  2.98256772e+03 -9.08829142e+02 -3.27660304e+03
  4.75031600e+03  9.09974184e+03 -7.89776916e+03 -1.27237185e+03
 -2.16116344e+03 -3.97307960e+03  5.62386980e+03 -6.44691548e+03
 -3.63198089e+03 -3.93034271e+03  1.64131326e+03 -3.86821134e+02
  3.54589115e+03 -2.09120009e+03 -2.27738184e+03  1.54177456e+04
  1.38565608e+04  7.01655477e+02  3.53195335e+03 -2.15213559e+03
 -4.30326120e+03 -5.28766934e+03 -9.18868882e+03 -1.03710745e+04
  1.79527819e+03 -1.62896036e+04  4.05351915e+03  3.40448865e+03
 -2.70661751e+03 -1.04951736e+02  6.31724758e+03 -2.79134319e+03
  8.60012370e+02  8.67242916e+03 -4.88249164e+03 -3.03827365e+02
  1.09927825e+03  8.92811640e+02 -5.54538113e+03  4.84740661e+03
  4.81442019e+03 -8.

In [32]:
# Özelliklerin önem derecelerini yazdırma
print('numerical_features')
for i in range(len(numerical_features)):
    print(f"{numerical_features[i]}: {features_importances[i]}")

numerical_features
room: 143.25154218814725
livingroom: 0.0
area: 4869.878988548078
age: -3578.314054978353
floor: 872.8402076602742


In [33]:
print("Categorical Features")
for i in range(len(categorical_features)):
    for j in range(len(model.named_steps['preparation'].transformers_[1][1].categories_[i])):
        print(model.named_steps['preparation'].transformers_[1][1].categories_[i][j], features_importances[len(numerical_features) + j])

Categorical Features
adana -2628.5698842761885
afyonkarahisar 2349.94073757155
aksaray -2938.38543034592
amasya -864.3601992885228
ankara 4400.120354248412
antalya 3472.4918804525187
aydin 4722.214847189735
balikesir 1862.6637849741978
bartin 3179.5012454903435
batman 1905.3155336541847
bayburt -17612.44018656759
bilecik -231.7523228024895
bitlis 2982.5677190205115
bolu -908.829142488768
burdur -3276.6030400292443
bursa 4750.315995903346
canakkale 9099.741838433354
corum -7897.769160698779
denizli -1272.3718505664385
diyarbakir -2161.163435048717
duzce -3973.079601572116
edirne 5623.869800018797
elazig -6446.9154827457105
erzincan -3631.980889790976
erzurum -3930.3427120080823
eskisehir 1641.3132566665497
gaziantep -386.8211342167372
giresun 3545.891153773402
hatay -2091.200086248623
isparta -2277.3818374750294
istanbul 15417.745607293547
izmir 13856.560848152694
kahramanmaras 701.6554768840928
karabuk 3531.953352307459
karaman -2152.1355871418064
kastamonu -4303.261197847965
kayseri -

In [34]:
new_data = pd.DataFrame({
    'city': pd.Categorical(['manisa']),
    'district': pd.Categorical(['yunusemre']),
    'neighborhood': pd.Categorical(['guzelyurt']),
    'room': [3],
    'livingroom': [1],
    'area': [190],
    'age': [5],
    'floor': [7]
})

prediction = model.predict(new_data)
print(prediction)


[41684.13670075]


In [35]:
print(df[(df['city'] == 'manisa') & (df['district'] == 'yunusemre') & (df['neighborhood'] == 'guzelyurt') ]) # Belirtilen özelliklere sahip satırları yazdırma

        city   district neighborhood  room  livingroom  area  age  floor  \
3001  manisa  yunusemre    guzelyurt     3           1   190    5      7   

      price  
3001  47000  


In [43]:
# R^2 değerini tolerans ile hesaplama
# Tolerans, tahminlerin gerçek değerlere ne kadar yakın olması gerektiğini belirler
tolerance = 10000 # Tolerans değeri
def tolerance_r2(y_true, y_pred, tolerance):
    residuals = y_pred - y_true # Kalanları hesaplama
    residuals[np.abs(residuals)<= tolerance] = 0 # Sıfır toleransına göre kalanları sıfırlama
    ssr = np.sum(residuals**2) # Kalanların karelerinin toplamını hesaplama
    sst = np.sum((y_true - np.mean(y_true))**2) # Toplam kare hatayı hesaplama
    return 1 - ssr / sst # R^2 değerini döndürme # R^2 değeri, modelin ne kadar iyi tahmin yaptığını gösterir

# Toleranslı R^2 değerini hesaplama
# Bu fonksiyon, tahminlerin gerçek değerlere ne kadar yakın olması gerektiğini belirler
def tolerance_percentage_r2(y_true, y_pred, tolerance):
    residuals = y_pred - y_true # Kalanları hesaplama
    residuals[(np.abs(residuals) / y_true <= tolerance)] = 0 # Sıfır toleransına göre kalanları sıfırlama
    ssr = np.sum(residuals**2) # Kalanların karelerinin toplamını hesaplama
    sst = np.sum((y_true - np.mean(y_true))**2) # Toplam kare hatayı hesaplama
    return 1 - ssr / sst # R^2 değerini döndürme # R^2 değeri, modelin ne kadar iyi tahmin yaptığını gösterir

In [44]:
print(r2_score(y_test, y_pred)) # Normal R^2 değeri
print(tolerance_r2(y_test, y_pred, 10000)) # Toleranslı R^2 değeri
print(tolerance_percentage_r2(y_test, y_pred, 0.5)) # Tolerans yüzdesi ile R^2 değeri

0.5006983969561993
0.66105536532552
0.7950515292151344
